# Simple quantum algorithms

## Phase kickback

In this page, we'll cover a behaviour of controlled quantum gates known as "phase kickback". While simple, this effect is uniquely quantum, and in this page we'll explore ways it can give us a small edge over classical computers for some toy problems.

In later pages, we'll see how we can extend concepts from these toy algorithms to more serious quantum algorithms, including Shor's factoring algorithm, and Grover's search algorithm.

### Eigenvectors

You should already be familiar with eigenvectors and eigenvalues, but if not, you can read a nice introduction [here](https://www.khanacademy.org/math/linear-algebra/alternate-bases/eigen-everything/v/linear-algebra-introduction-to-eigenvalues-and-eigenvectors). If you _are_ familiar, then you should recognise the eigenvector equation:

$$ \class{_matrix-A}{A}\class{_eig-vec-A}{|x\rangle} = \class{_eig-val-A}{\lambda}\class{_eig-vec-A}{|x\rangle} $$

This is even simpler in quantum computing. Since all our state vectors have a magnitude of 1, our eigenvalues also need to have a magnitude of 1, i.e. $\lambda = e^{2\pi i \theta}$. So for a quantum gate $U$ and its eigenstate $|x\rangle$, we have:

$$ \class{_matrix-U}{U}\class{_eig-vec-U}{|x\rangle} = \class{_eig-val-U}{e^{2\pi i \theta}}\class{_eig-vec-U}{|x\rangle} $$

To sum up: If a gate rotates (and only rotates) all the amplitudes of a state vector by the same amount, then that state is an _eigenstate_ of that gate.

### Controlled gates & eigenstates

Once you're comfortable with the concept of eigenstates, we can start to think about what happens when we control these circuits on the state of another qubit. For example, we know the Z-gate acting on the state $|1\rangle$ introduces a negative global phase ($\theta = 0.5$).

$$ Z|1\rangle = -|1\rangle $$

Let's work out what happens when we control this operation.

<!-- ::: q-block.tabs -->

### The controlled-Z gate

<!-- ::: tab -->

### |10〉

<!-- ::: column(width=200) -->

If the control qubit is $|0\rangle$, then the behaviour is trivial; nothing happens.

<!-- ::: column(width=400) -->

![circuit diagram showing a two-qubit register in the state 01, acted on by a CZ-gate. The state is unchanged](images/kickback/tabs/0/0.svg)

<!-- ::: -->

<!-- ::: tab -->

### |11〉

<!-- ::: column(width=200) -->

If the control qubit is $|1\rangle$, the gate introduces a global phase (note the minus sign in the image to the right), but the qubit's states are unchanged.

<!-- ::: column(width=400) -->

![circuit diagram showing a two-qubit register in the state 01, acted on by a CZ-gate. The state is unchanged](images/kickback/tabs/0/1.svg)

<!-- ::: -->

<!-- ::: tab -->

### |1+〉

<!-- ::: column(width=200) -->

The controlled-Z gate does nothing when the control is $|0\rangle$, and introduces a negative phase when the control is $|1\rangle$. When the control qubit is in superposition, the gate changes the _relative_ phase between the $|0\rangle$ and $|1\rangle$ states of the control qubit.

<!-- ::: column(width=400) -->

![circuit diagram showing a two-qubit register in the state 01, acted on by a CZ-gate. The state is unchanged](images/kickback/tabs/0/2.svg)

<!-- ::: -->

<!-- ::: -->

<!-- ::: -->

When the control is $|{+}\rangle$, and the target is $|1\rangle$, the controlled-Z gate changes the state of the _control_ qubit, but leaves the target qubit unchanged. This effect is called "phase kickback", since the eigenvalue makes its way back into the state of the control qubit.

More generally, If we have a quantum gate $U$, and it's eigenstate $|x\rangle$, then $U$ acting on $|x\rangle$ will add a global phase $\theta$ as we saw in the eigenvector equation (shown again below).

$$ \class{_matrix-U}{U}\class{_eig-vec-U}{|x\rangle} = \class{_eig-val-U}{e^{2\pi i \theta}}\class{_eig-vec-U}{|x\rangle} $$

If we control the operation $U|x\rangle$ by another qubit in a superposition of $|0\rangle$ and $|1\rangle$, then this will have the effect of rotating the control qubit around the Z-axis by angle $\theta$. I.e.:

$$ \class{_matrix-CU}{CU}\class{_eig-vec-U}{|x\rangle}\class{_control-qubit-pre}{(\alpha|0\rangle + \beta|1\rangle)} = \class{_eig-vec-U}{|x\rangle}\class{_control-qubit-post}{(\alpha|0\rangle + \beta e^{2\pi i \theta}|1\rangle)} $$

![](images/kickback/general-kickback.svg)

In the example above, we see that the 'control' of the controlled-Z gate is actually doing a Z-rotation - something that should have only been observing the qubit has actually changed it.

### The CNOT Gate

Let's look at a different specific case of phase kickback. Since the $|{-}\rangle$ state is an eigenstate of the X-gate (with eigenvalue $-1$), we get:

$$ \class{_matrix-CX}{CX}\class{_eig-vec-X}{|{-}\rangle}\class{_control-qubit-pre}{(\alpha|0\rangle + \beta|1\rangle)} = \class{_eig-vec-X}{|{-}\rangle}\class{_control-qubit-post}{(\alpha|0\rangle - \beta |1\rangle)} $$

![](images/kickback/cnot-kickback.svg)

Again, in this case the phase change $\theta = 0.5$, so our control qubit is flipped around the Z-axis.

When we remember that the H-gate does the transformation $|0\rangle \rightarrow |{+}\rangle$ and $|1\rangle \rightarrow |{-}\rangle$ (and vice versa), we get the following identity:

![](images/kickback/cnot-identity.svg)

## Deutsch's problem

We've just seen that conditioning an action on the state of a qubit can actually change the state of the controlling qubit. This is a uniquely quantum effect, i.e. something we don't see happen with classical bits.

In quantum computing, we want to create algorithms that classical computers _can't_ carry out, so a good place to start is to try and re-frame this effect as a problem to be solved. This way, we can prove quantum computers are at least slightly better at something than classical computers.

Deutsch's problem does exactly this. Deutsch's is a 'black box' problem: An artificial problem where we're allowed to apply a function to our bits, but we're not allowed to look at how the function works. The challenge is to discover some properties of the box by trying different inputs and observing the outputs.

Deutsch's problem is as follows: We're given a classical, reversible function, (which we'll call $f$), that acts on two bits, $a$ & $b$. The function will leave bit $a$ alone, but may, or may not flip bit $ b $. Deutsch's problem asks us to work out whether $f$ behaves differently depending on the value of $a$ (we'll call this "balanced" behaviour), or if it ignores $a$ and always does the same thing to $b$ ("constant" behaviour). The challenge is to discover this while applying $f$ as few times as possible.

![Image of deutsch's problem as a quantum circuit. The circuit has two bits, the top and bottom wires labelled 'a' and 'b' respectively. Both bits are then acted on by an opaque, two-bit gate labelled 'f'. After 'f', the top wire is still labelled 'f', but the bottom wire is now labelled 'f(a, b)'.](images/kickback/deutsch-problem.svg)

The best classical algorithm for this problem applies $f$ twice with different values of $a$, then looks to see if $f$ behaved differently.

## Deutsch's algorithm

As you may have guessed, we can use phase kickback to create a quantum algorithm that does even better than the classical algorithm. If we put qubit $ a $ in the $|{+}\rangle$ state, and qubit $ b $ in the $|{-}\rangle$ state, then any flip conditioned on $ a $ will kick back a negative relative phase, flipping qubit $ a $ from $|{+}\rangle$ to $|{-}\rangle$. We can then apply a H-gate to $ a $ to see whether kickback occurred or not. This means we only need to apply $f$ once to get our answer.

![Image of deutsch's algorithm as a quantum circuit.](images/kickback/deutsch-algorithm.svg)

<!-- ::: q-block.reminder -->

### More info

<details>
    <summary>Inside the black box (click to expand)</summary>
If this still seems like magic, it can help to think about all the possible Deutsch functions and the quantum circuits that implement them. There are four possible Deutsch functions: two constant, and two balanced.

If constant, the function can either do nothing, or flip qubit $ b $. If balanced, the function can either flip $ b $ only when $ a $ is $|1\rangle$, or flip $ b $ only when $ a $ is $|0\rangle$. You can see all four scenarios in the image below.

<img src="images/kickback/deutsch-oracles.svg">

With both constant functions, the topmost qubit will remain unchanged (since we're not doing anything to it), and in the balanced functions, the kickback effect flips the topmost qubit from $|{+}\rangle$ to $|{-}\rangle$.
</details>

<!-- ::: -->

This isn't the most impressive example of quantum speedup; it's very specific, and we don't find black box problems in the wild. Instead, Deutsch's problem gives us an encouraging result, and some interesting effects to explore. In the rest of this course, we'll extend this simple experiment to solve even more impressive problems, including factoring.



<!-- ::: q-block.exercise -->

### Exercise

Make a function, `deutsch()` that takes a Deutsch function as a `QuantumCircuit` and uses the Deutsch algorithm to solve it on a quantum simulator. Your function should return `True` if the Deutsch function is balanced, and `False` if it's constant.

You can use the provided function `deutsch_problem()` to create a `QuantumCircuit` to use as input to your `deutsch()` function.

<!-- ::: -->

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

def deutsch_problem(seed=None):
    """Returns a circuit that carries out the function
    from Deutsch's problem.
    Args:
        seed (int): If set, then returned circuit will
            always be the same for the same seed.
    Returns: QuantumCircuit
    """
    np.random.seed(seed)
    problem = QuantumCircuit(2)
    if np.random.randint(2):
        print("Function is balanced.")
        problem.cx(0, 1)
    else:
        print("Function is constant.")
    if np.random.randint(2):
        problem.x(1)
    return problem

In [ ]:
def deutsch(function):
    """Implements Deutsch's algorithm.

    Args:
        function (QuantumCircuit): Deutsch function to solve.
            Must be a 2-qubit circuit, and either balanced,
            or constant.
    Returns:
        bool: True if the circuit is balanced, otherwise False.
    """

    # your code here

## Extending Deutsch's problem

Here, we'll explore two different ways of extending Deutsch's problem to be slightly more difficult for classical computers. Both these extensions generalize the controlling conditions to work on $n$ bits instead of one, and as such make the task of checking behaviour on different inputs slightly harder.

### The Bernstein Vazirani problem

In this problem we're again asked to discover some behaviour of a black-box, but this time the box takes a string of $n$ input bits (we'll call this string $\mathbf{x}$), and outputs $1$ bit. In simple terms: For each input bit $x_i \in \mathbf{x}$, the box will either:

- a) Flip the output bit if $x_i = 1$, or
- b) do nothing

Our task is to work out which input bits trigger the flip.

Equivalently, but more formally, we can say the black-box performs the function:

$$ f(\mathbf{x}) = \mathbf{s} \cdot \mathbf{x} \pmod 2 $$

where the dot ($\cdot$) represents the [bitwise](gloss:bitwise) product. This box takes $n$ controlling bits (we'll call this the input register), and writes the output to a single target bit. Our task is to work out $\mathbf{s}$.

<img src="images/kickback/bv-problem.svg">

The code cell below implements an instance of the Bernstein-Vazirani function in Python. Can you write some code that works out `s`, only by calling the function? How many times do you need to call it?

In [64]:
def bv_function(x):
    """Implements the Bernstein-Vazirani function (x.s mod 2).
    Args:
       x (str): Input string, must only contain 1s and 0s
    Returns:
       int: Result of x.s mod 2; either 0 or 1
    """
    s = '0011101'  # challenge is to find this string
    if len(s) != len(x):  # quick sanity test
        raise ValueError(f'Input must have {len(s)} bits')
    output = 0  # now do bitwise product of 'x' and 's'
    for i in range(len(x)):
        output += int(x[i]) * int(s[i])
    return output % 2  # output product mod 2

# Try changing the input to the function call below
bv_function('0000000')

0

<!-- vale QiskitTextbook.Spelling = NO -->


<!-- ::: q-block.reminder -->

### Classical solution

<details>
    <summary>Click to reveal the best classical algorithm for the Bernstein-Vazirani problem.</summary>
    We can isolate each bit of $\mathbf{s}$ by passing an input string with exactly one `1` in it (and all other bits set to `0`).
    The best classical algorithm is to try out each of these $n$ input strings, and see what happens to the output. The function below takes a Bernstein-Vazirani function (as defined above) and outputs $\mathbf{s}$.
    
    ```{python}
    def solve_bv_function(func, n=7):
        # takes a Bernstein-Vazirani function
        # and the length of `s`. Outputs `s`.
        s = ''
        for i in range(n):
            x = '0'*i + '1' + '0'*(n-i-1)
            out = func(x)
            s += str(out)
            print(f'f({x}) = {out}')
        return s
    ```
</details>

<!-- ::: -->


<!-- vale QiskitTextbook.Spelling = YES -->

### Quantum solution

The simple way of phrasing the Bernstein-Vazirani problem is: "Some input bits affect the output bit, others do not. Which bits are which?". When put like this, we can see the Bernstein-Vazirani problem is essentially $n$ Deutsch problems at once (and if $n = 1$, it _is_ Deutsch's problem). As such, the best quantum algorithm is almost identical to Deutsch's algorithm:
1. Set the qubit that stores the output to $|{-}\rangle$, and set all the input qubits to $|{+}\rangle$. 
2. Apply the function
3. Apply Hadamard gates to the input register again

Measuring the input register after this process will tell us $\mathbf{s}$.

<img src="images/kickback/bv-algorithm.svg">

Any qubits that flip the output will have a negative phase kicked back to them, and so will be $|1\rangle$ when we come to measure them. Any qubits that have no effect on the output will measure as $|0\rangle$.

### The Deutsch-Jozsa problem

In this extension of Deutsch's problem, the black-box cannot necessarily be broken into bitwise operations. Instead, we're guaranteed the box will either:

- a) Output the same value, regardless of the input state (we'll call this behaviour _constant_), or
- b) output `0` on exactly half the input states, and `1` for the rest (we'll call this _balanced_)

Once again, our job is to work out if the box is constant or balanced. The code cell below implements the Deutsch-Jozsa function in Python.

In [92]:
def dj_function(x, seed=1):
    """Implements a Deutsch-Jozsa function.
    Args:
       x (str): Input string, must only contain 1s and 0s
    Returns:
       int: Result of box (either 0 or 1)
    """
    import random
    random.seed(seed)  # fix results
    if random.randint(0, 1) == 1:
        # function is constant
        return random.randint(0, 1)
    
    # function is balanced
    n = 2**len(x)
    possible_inputs = [f'{s:07b}' for s in range(n)]
    flip_strings = random.sample(possible_inputs, n//2)
    if x in flip_strings:
        return 1
    else:
        return 0

# Try changing the input to the function call below
dj_function('0000000')

1

### Quantum solution

Again, we're asked to work out whether an operation depends on our input bits or not, and again the solution involves a load of Hadamard gates.

Earlier, we viewed the Bernstein-Vazirani problem as $n$ simultaneous Deutsch problems. Here, we can view the Deutsch-Jozsa problem as Deutsch's problem acting on an entire register instead of a single qubit.

We can sort each possible input state into one of two groups:
- States that flip the output qubit
- States that do _not_ flip the output qubit

So any state of our quantum register is of the form:

$$|\psi\rangle =  \alpha|\text{flip}\rangle + \beta|\text{noflip}\rangle $$

where $|\text{flip}\rangle$ is any state (or superposition of states) that flips the output bit, and $|\text{noflip}\rangle$ is any quantum state that leaves the output bit unchanged. Hopefully you can already see how the solution is going to work.

$$
\begin{aligned}
|\psi\rangle|{-}\rangle &=  \tfrac{1}{\sqrt{2}}|\text{flip}\rangle|{-}\rangle + \tfrac{1}{\sqrt{2}}|\text{noflip}\rangle|{-}\rangle \\
U_f|\psi\rangle|{-}\rangle &=  \tfrac{1}{\sqrt{2}}|\text{flip}\rangle|{-}\rangle - \tfrac{1}{\sqrt{2}}|\text{noflip}\rangle|{-}\rangle
\end{aligned}
$$